#The Spiking Neuron Model - Coding Challenge Problems (Part 2)


# Extending the Model: Spike-Rate Adaptation
This interactive document is meant to be followed as the reader makes their way through chapter: *The Spiking Neuron Model*.  Each model presented in the chapter will have a section consisting of a step-by-step walkthrough of a simple Python implementation.  This is followed by an interface to run simulations with different parameter values to answer the Coding Challenge Problems.

For each model covered in the chapter, there is a section called **Simulations for Coding Challenge Problems.**  This is where you will find user-interface components such as value sliders for various parameters.  Use these controls to answer the questions from the text, and feel free to explore how the values affect the results.


**Content creators**: Maxwell E. Bohling, Dr. Lawrence Udeigwe

## How This Works

This Jupyter Notebook has both Content cells and Code cells.  As you make your way through the tutorial, you MUST make sure to run each code cell as you come to them.  Each code cell has a *Play* button next to it which will execute the code. Some code may be hidden (generally because it is more complex code not completely necessary to understand for the purposes of the chapter Coding Challenge Problems.)

**IMPORTANT**: You are currently viewing a copy of the original notebook.  You will find that you can edit the content of any cell.  If you accidently change a cell, such as a line of code and/or run into errors as you try to run subsequent blocks, simply refresh the page, OR go to the *Runtime* Menu and select *Restart runtime*.  It is also suggested that you go to the *Edit* menu and select *Clear all outputs.*  This will always allow you to revert the notebook to the original version (though you will have to run each code block again.)



 Execute the code block. **Initialize Setup**

In [ ]:
#@title Initialize Setup
#@markdown Import python libraries, and define helper functions (No need to understand this code, simply make sure you run this first.)
import sys
import functools as ft
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy as sc

def printError():
  message = 'Something went wrong!\n\n'  
  message = message + 'Check for the following:\n\n'
  message = message + '\t1.  Variables are set to the correct values.  Make sure to replace any instances of \'None\'.\n'
  message = message + '\t2.  All previous code blocks have been run the order they appear and output a success message.\n'
  message = message + '\t3.  No other code has been altered.\n\n'
  message = message + 'then try running the code block again.\n'
  print(message)

def printSuccess():
  message = 'Success!  Move on to the next section.'
  print(message)

def checkVoltageParameters(Vrest, Vth, Vreset, Vspike):
  # Check: IAF-Voltage-Parameters
  print('Checking Voltage Parameters...\n')
  try:
      check_Vrest = Vrest
      check_Vth = Vth
      check_Vreset = Vreset
      check_Vspike = Vspike
  except:
    printError()
  else:
    vals = [Vrest, Vth, Vreset, Vspike]
    correct_vals = [-70, -54, -80, 0]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()
 
def checkNeuronProperties(Rm, Cm, tau_m):
  # Check: IAF-Neuron-Properties
  print('Checking Neuron Properties...\n')
  try:
      check_Rm = Rm
      check_Cm = Cm
      check_tau_m = tau_m
  except:
    printError()
  else:
    vals = [Rm, Cm, tau_m]
    correct_vals = [10, 1, 10]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()

def checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, Ie, t_pulse, start_current, end_current):
  # Check: IAF-Simulation-Setup
  print('Checking Simulation Setup...\n')
  try:
    check_Vrest = Vrest
    check_Vinitial = Vinitial
    check_EL = EL
    check_t0 = t0
    check_dt = dt
    check_t_final = t_final
    check_time = time
    check_Ie = Ie
    check_t_pulse = t_pulse
    check_start_current = start_current
    check_end_current = end_current
  except:
    printError()
  else:
    vals = [Vrest, Vinitial, EL, t0, dt, t_final, Ie, t_pulse, start_current, end_current]
    correct_vals = [-70, -70, -70, 0, 1, 500, 1.75, 300, 100, 400]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      if len(time) == 500 and time[0] == 0 and time[-1] == 499:
        printSuccess()
      else:
        printError()
    else:
      printError()

def autoSetValues():
  Vrest = -70
  Vth = -54 
  Vreset = -80
  Vspike = 0
  Vinitial = Vrest
  EL = Vrest
  t0 = 0
  dt = 1
  t_final = 500
  time = range(t0, t_final, dt)
  t_pulse = 300 
  start_current = np.absolute(t_final-t_pulse)/2 
  end_current = start_current+t_pulse 
  V = [0] * len(time)
  V[0] = Vrest

try:
  check_sys = sys
except:
  printError()
else:
  modulename = 'functools'
  if modulename not in sys.modules:
    printError()
  else:
    printSuccess()

## Walkthrough
The goal of this section is to extend the integrate-and-fire neuron model such that our simulation more realistically resembles neural spiking behavior.  

In particular, we want to account for a biological phenomenon observed in spiking neurons called spike-rate adaptation, or the reduction of a neuron's ability to spike while undergoing a stimulus of constant intensity.

### Membrane Equation
Recall that the basic equation is expressed as follows:

> $ \displaystyle \tau_{m}\frac{dV}{dt} = E_{L} - V(t) + r_{m}g_{sra}(V-E_K) + R_{m}I_{e} $


The parameter $g_{sra}$ is the spike-rate adaptation conductance that increases at each spike by an amount $dg_{sra}$.  This term serves as the mechanism that increases *refractoriness* as a neuron experiences repetitive spiking.  The $g_{sra}$ conductance adheres to the following rules:

1. At each spike occurence, $g_{sra}$  = $g_{sra}$ + $dg_{sra}$.
2. In the absence of spikes, the amount $dg_{sra}$ decays exponentially to 0 according to the time constant $\tau_{sra}$, according to:

> $ \displaystyle \tau_{sra}\frac{dg_{sra}}{dt} = -g_{sra}$  

### Voltage Parameters

As in the integrate-and-fire walk through, the *Voltage Parameters* that determine the resting, threshold, spike, and reset values:

*   $ V_{rest} = -70\;$*mV*
*   $ V_{th} = -54\;$*mV*
*   $ V_{reset} = -80\;$*mV*
*   $ V_{spike} = 0\;$*mV*







### Neuron Properties
As in the integrate-and-fire walkthrough, we now need to define the values for the Membrane Resistance $ R_{m} $, the Membrane Capacitance, $ C_{m} $, and calculate the Membrane Time Constant $\tau_{m}$.

To incorporate spike-rate adaptation, we must explicitly define additional parameters including the neuronal surface area $A$, specific membrane resistance $r_m$, and properties necessary to compute the evolution of the spike-rate adaptation conductance $g_{sra}$; the amount of change per unit-time of the conductance throughout the simulation $dg_{sra} $, and $\tau_{sra}$ which controls the speed at which the conductance evolves.

*   $ A = 0.1\;$*mm*$^2$
*   $ r_{m} = 1\;M\Omega \times\;$*mm*$^2$
*    $ \displaystyle R_{m} = \frac{r_m}{A} = \frac{1}{0.1} = 10\;M\Omega $
*    $ C_{m} = 1\;$*nF*
*    $ \tau_{m} = R_{m}C_{m} = 10\;$*ms*
*    $dg_{sra} = 0.1\;$*nF*
*    $\tau_{sra} = 150\;$*ms*



### Simulation Setup & Computing and Storing $\frac{dV}{dt}$, $\frac{dg_{sra}}{dt}$
We setup our simulation identically to how it was setup in the integrate-and-fire walkthrough, with the exception of an initial value for $g_{sra}$, a list to hold the values of  $g_{sra}(t)$, and the reversal potential corresponding to the spike-rate-adaptation conductance, $E_{K}$  (thus, this conductance is modeled as a K$^+$ (potassium) conductance).

* $g_{sra} = 0\;$*nF*
* $E_{K} = -75\;$*mV*

In [ ]:
try:
  # Voltage Paramaters - Units mV (1 mV = 1e-3 Volts)
  Vrest = -70
  Vth = -54 
  Vreset = -80
  Vspike = 0
  # Neuron Surface Area - Units mm^2
  A = 0.1
  # Specific Membrane Resistant - Units M_Ohm * mm^2
  rm = 1
  # Total Membrane Resistance - Units M_Ohm (1 M_Ohm = 1e6 Ohms)
  Rm = rm/A
  # Total Membrane Capacitance - Units nF
  Cm = 1
  # Membrane Time Constant - Units ms (1 ms = 1e-3 Seconds)
  tau_m = Rm*Cm
  # Spike-Rate Adaptation Time Constant - Units ms
  tau_sra = 150
  # Initial voltage
  Vinitial = Vrest
  # Leakage conductance reversal potential
  EL = Vrest
  # Simulation time span (0 to 500ms, dt = 1ms)
  t0 = 0
  dt = 1
  t_final = 500
  time = range(t0, t_final, dt)
  # Current pulse time span - Units ms (1 ms = 1e-3 Seconds)
  t_pulse = 300 
  # Time at which the current is applied
  start_current = np.absolute(t_final-t_pulse)/2 
  # Time at which the current is switched off
  end_current = start_current+t_pulse
  # Input current: Ie - Units nA (1 nA = 10-9 Amperes)
  Ie = 1.75
  # Create a list V(t) to store the value of V at each time-step dt
  V = [0] * len(time)
  # Set the initial value at time t = t0 to the initial value Vinitial
  V[0] =  Vinitial

  # K+ Conductance reversal potential
  EK =-75

  # Initial spike-rate-adaptation values
  g_sra = 0
  dg_sra = 0.1
  tau_sra = 300

  # Create a list Gsra(t) to store the value of g_sra at each time-step dt
  Gsra = [0] * len(time)

  # Set the initial value at time t = t0 to the initial value g_sra
  Gsra[0] =  g_sra
except:
  printError()
else:
  printSuccess()

Finally, we run our simulation according to the updated *pseudocode*

---

*for each time-step from $t = t_{0}$ to $t = t_{final}$*
> *If the current time $t < start_{current}\ $ or $\ t > end_{current}$*
>> $I_{e} = 0$

> *otherwise*
>> $I_{e} = 1.75\ nA$

> *Use Euler's Method of Numerical Integration*
> $\ \displaystyle dV = \frac{dt}{\tau_{m}}(E_{L} - r_{m}g_{sra}(V(t) - E_{K}) +  R_{m}I_{e})$

> *Update* $V(t+1) = V(t) + dV$

> *If $\ V(t+1) \geq V_{th} $* 
>> $V(t) = V_{spike}$

>> $V(t+1) = V_{reset}$

>> $g_{sra}(t+1) = g_{sra}(t) + dg_{sra}$

>*else*
>> $ \displaystyle g_{sra}(t+1) = \frac{dt}{\tau_{sra}}(-g_{sra}(t)) $

*end*

---

This translates to the following Python code.

In [ ]:
try:
  # For each timestep we compute V and store the value
  for t in time[0:-2]:

      # If time t < 100 ms, we do not turn the Injected Current on yet.
      # If time t > 400ms, we turn the Injected Current off.
      if t < start_current or t > end_current:
          ie = 0
      # Otherwise, we switch on the Injected Current 
      else:
          ie = Ie

      # Using Euler's Method for Numerical Integration (See Chapter Text)
      # we compute the change in voltage dV as follows (using the model equation)
      dV = (dt/tau_m)*(EL - V[t] - (rm*Gsra[t]*(V[t] - EK)) + Rm*ie)

      # Store this new value into our list
      V[t+1] = V[t] + dV

      # If the voltage value we computed crosses the threshold value Vth, a spike occurs.
      if V[t+1] >= Vth:
          # In the event of a spike, the membrane potential spikes up to Vspike mV
          V[t] = Vspike
          # Followed by immediate hyperpolarization to its Vreset value.
          V[t+1] = Vreset

          # In the event of a spike, the spike-rate adaptation conductance increases by dg_sra
          Gsra[t+1] = Gsra[t] + dg_sra
      else:
        # Using Euler's Method for Numerical Integration (See Chapter Text)
        Gsra[t+1]=  Gsra[t] + (dt/tau_sra)*(-Gsra[t])
except:
  printError()
else:
  printSuccess()

### Visualizing Results

Now we have values of $V$ for each time-step of the simulation, we can visualize the results by using Python to plot the data.  This makes use of another widely used library **matplotlib**.

In [ ]:
try:
  check_plt = plt
except:
  printError()
else:
  # Plot the membrane potential and the spike-rate adaptation conductance
  fig, axs = plt.subplots(nrows=2, ncols=1, sharex= True)
  axs[0].plot(time[0:-2], V[0:-2])
  axs[0].set_title('Spike-rate Adaptation: $V$ against Time')
  axs[0].set_xlabel('Time (ms)')
  axs[0].set_ylabel('$V(t)$')
  axs[0].hlines(-54,time[0],time[-2],colors='r',linestyles='dashed')

  axs[1].plot(time[0:-2], Gsra[0:-2])
  axs[1].set_title('$g_{sra}$ against Time')
  axs[1].set_xlabel('Time (ms)')
  axs[1].set_ylabel('$g_{sra}\;$nF')

  # Display the plot
  plt.show()    

## Full Integrate-and-Fire Model Neuron Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Voltage Paramaters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70
Vth = -54 
Vreset = -80
Vspike = 0

# Neuron Surface Area - Units mm^2
A = 0.1

# Specific Membrane Resistant - Units M_Ohm * mm^2
rm = 1

# Total Membrane Resistance - Units M_Ohm (1 M_Ohm = 1e6 Ohms)
Rm = rm/A

# Total Membrane Capacitance - Units nF
Cm = 1

# Membrane Time Constant - Units ms (1 ms = 1e-3 Seconds)
tau_m = Rm*Cm

# Spike-Rate Adaptance Parameters - Units mV (1 mV = 1e-3 Volts)
g_sra = 0
dg_sra = 0.1

# Spike-Rate Adaptation Time Constant - Units ms
tau_sra = 100

# Initial V
Vinitial = Vrest

# (Equilibrium/Reversal Potential = Resting Potential)
EL = Vrest

# (Equilibrium/Reversal Potential = Resting Potential for K+)
EK = -75

# Simulation Time Span (0 to 500ms, dt = 1ms)
t0 = 0
dt = 1
t_final = 500

# Create the list of time step values
time = range(t0, t_final, dt)

# Current Pulse Time Span - Units ms (1 ms = 1e-3 Seconds)
t_pulse = 300 

# Time at which the current is applied
start_current = np.absolute(t_final-t_pulse)/2 

# Time at which the current is switched off
end_current = start_current+t_pulse 

# Input Current: Ie - Units nA (1 nA = 10-9 Amperes)
# Using 1.75 for default
Ie = 1.75

# Create a list V(t) to store the value of V at each time-step dt
V = [0] * len(time)

# Set the initial value at time t = t0 to the resting potential
V[0] = Vinitial

# Create a list Gsra(t) to store the value of g_sra at each time-step dt
Gsra = [0] * len(time)

# Set the initial value at time t = t0 to the initial value g_sra
Gsra[0] =  g_sra

# For each timestep we compute V and store the value
for t in time[0:-2]:

    # If time t < 100 ms, we do not turn the Injected Current on yet.
    # If time t > 400ms, we turn the Injected Current off.
    if t < start_current or t > end_current:
        ie = 0
    # Otherwise, we switch on the Injected Current 
    else:
        ie = Ie

    # Using Euler's Method for Numerical Integration (See Chapter Text)
    # we compute the change in voltage dV as follows (using the model equation)
    dV = (dt/tau_m)*(EL - V[t] - (rm*Gsra[t]*(V[t] - EK)) + Rm*ie)

    # Store this new value into our list
    V[t+1] = V[t] + dV

    # If the voltage value we computed crosses the threshold value Vth, a spike occurs.
    if V[t+1] >= Vth:
        # In the event of a spike, the membrane potential spikes up to Vspike mV
        V[t] = Vspike
        # Followed by immediate hyperpolarization to its Vreset value.
        V[t+1] = Vreset

        # In the event of a spike, the spike-rate adaptation conductance increases by dg_sra
        Gsra[t+1] = Gsra[t] + dg_sra
    else:
      # Using Euler's Method for Numerical Integration (See Chapter Text)
      Gsra[t+1]=  Gsra[t] + (dt/tau_sra)*(-Gsra[t])


# Plot the membrane potential and the spike-rate adaptation conductance
fig, axs = plt.subplots(nrows=2, ncols=1, sharex= True)
axs[0].plot(time[0:-2], V[0:-2])
axs[0].set_title('Spike-rate Adaptation: $V$ against Time')
axs[0].set_xlabel('Time (ms)')
axs[0].set_ylabel('$V(t)$')
axs[0].hlines(-54,time[0],time[-2],colors='r',linestyles='dashed')

axs[1].plot(time[0:-2], Gsra[0:-2])
axs[1].set_title('$g_{sra}$ against Time')
axs[1].set_xlabel('Time (ms)')
axs[1].set_ylabel('$g_{sra}\;$nF')

# Display the plot
plt.show()  

## Simulations for Coding Challenge Problems